In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from numpy.random import randn, seed
import scipy.stats
from scipy.optimize import minimize

In [2]:
data = pd.read_excel("./PreciosPortf.xlsx", index_col=0)
rend = data.pct_change().dropna()

rend.head()

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
DATE,,,,,
2016-01-04,-0.022961,-0.033688,-0.007448,-0.014946,0.008847
2016-01-05,-0.022074,-0.000763,0.003127,-0.004202,-0.020554
2016-01-06,-0.004549,-0.000981,-0.021366,-0.006564,-0.007555
2016-01-07,-0.005432,-0.022809,0.001506,-0.056395,0.019312
2016-01-08,-0.004118,-0.007706,-0.003180,0.026007,-0.017148


In [3]:
def port_rend(weights, r):
    E_p =(r.mean() @ weights) *252
    return E_p
def port_vol(weights, r):
    S_p = np.sqrt(weights.T @ (r.cov() *252) @ weights)
    return S_p

In [4]:
def sum_weights(weights):
    return weights.sum() -1 # == 0

In [5]:
rf = 0.06
N, M  = rend.shape
np.random.seed(5)
w0 = np.random.randn(M)
def min_func_sharpe(weights):
        return - (port_rend(weights, rend) - rf)/ port_vol(weights,rend)

constraints = [
    {"type":"eq", "fun":sum_weights}
]
f_sharpe = minimize(min_func_sharpe,w0, constraints=constraints)


In [6]:
w_sharpe = f_sharpe.x
w_sharpe

array([ 0.73100198, -0.14073714,  0.17604817,  0.16495151,  0.06873549])

In [7]:
Nocional = 10_000_000

In [8]:
vector_posiciones = w_sharpe*Nocional
vector_posiciones

array([ 7310019.76031335, -1407371.40209152,  1760481.71748185,
        1649515.05054508,   687354.87375123])

In [9]:
volatilidad_diaria = np.sqrt(np.diagonal(rend.cov()))
volatilidad_diaria

array([0.01495963, 0.01847597, 0.02340559, 0.01466858, 0.01501933])

In [10]:
volatilidad_anual = volatilidad_diaria*np.sqrt(252)
volatilidad_anual

array([0.23747683, 0.29329688, 0.37155226, 0.23285656, 0.23842444])

In [11]:
q_95 = scipy.stats.norm.ppf(0.95)
q_99 = scipy.stats.norm.ppf(0.99)
q_95, q_99

(1.6448536269514722, 2.3263478740408408)

In [12]:
vector_posiciones

array([ 7310019.76031335, -1407371.40209152,  1760481.71748185,
        1649515.05054508,   687354.87375123])

In [13]:
var_individual_diario_95 = (q_95*vector_posiciones*volatilidad_diaria).sum()
var_individual_diario_99 = (q_99*vector_posiciones*volatilidad_diaria).sum()
var_individual_diario_95, var_individual_diario_99

(261659.12261559494, 370069.4904678832)

In [14]:
var_individual_anual_95 = (q_95*vector_posiciones*volatilidad_anual).sum()
var_individual_anual_99 = (q_99*vector_posiciones*volatilidad_anual).sum()
var_individual_anual_95, var_individual_anual_99

(4153709.800273325, 5874671.037542442)

In [15]:
VaR_anual_95 = q_95*Nocional*port_vol(w_sharpe, rend)
VaR_anual_99 = q_99*Nocional*port_vol(w_sharpe, rend)
VaR_anual_95,VaR_anual_99

(3257144.4803131097, 4606641.59598428)

In [16]:
port_vol(w_sharpe, rend)

0.19802032393300636

In [17]:
VaR_diario_95 = q_95*Nocional*port_vol(w_sharpe, rend)/np.sqrt(252)
VaR_diario_99 = q_99*Nocional*port_vol(w_sharpe, rend)/np.sqrt(252)
VaR_diario_95, VaR_diario_99

(205180.81616940963, 290191.14386143064)

In [18]:
Efecto_Diversificacion_Diario_95 = var_individual_diario_95 - VaR_diario_95
Efecto_Diversificacion_Diario_99 = var_individual_diario_99 - VaR_diario_99
Efecto_Diversificacion_Diario_95, Efecto_Diversificacion_Diario_99

(56478.30644618531, 79878.34660645254)

In [19]:
Efecto_Diversificacion_Anual_95 = var_individual_anual_95 - VaR_anual_95
Efecto_Diversificacion_Anual_99 = var_individual_anual_99 - VaR_anual_99
Efecto_Diversificacion_Anual_95, Efecto_Diversificacion_Anual_99

(896565.3199602151, 1268029.4415581618)

In [20]:
print(f""" ## VaR del Portafolio ##
VaR Anual al 95% : ${round(VaR_anual_95,2):,}
VaR Anual al 99% : ${round(VaR_anual_99,2):,}
VaR Diario al 95% : ${round(VaR_diario_95,2):,}
VaR Diario al 99% : ${round(VaR_diario_99,2):,}

## VaR Individual ##
VaR Individual Anual al 95% : ${round(var_individual_anual_95,2):,}
VaR Individual Anual al 99% : ${round(var_individual_anual_99,2):,}
VaR Individual Diario al 95% : ${round(var_individual_diario_95,2):,}
VaR Individual Diario al 99% : ${round(var_individual_diario_99,2):,}

## Efecto de Diversificación ##
VaR Individual Anual al 95% : ${round(Efecto_Diversificacion_Anual_95,2):,}
VaR Individual Anual al 99% : ${round(Efecto_Diversificacion_Anual_99,2):,}
VaR Individual Diario al 95% : ${round(Efecto_Diversificacion_Diario_95,2):,}
VaR Individual Diario al 99% : ${round(Efecto_Diversificacion_Diario_99,2):,}
""")

 ## VaR del Portafolio ##
VaR Anual al 95% : $3,257,144.48
VaR Anual al 99% : $4,606,641.6
VaR Diario al 95% : $205,180.82
VaR Diario al 99% : $290,191.14

## VaR Individual ##
VaR Individual Anual al 95% : $4,153,709.8
VaR Individual Anual al 99% : $5,874,671.04
VaR Individual Diario al 95% : $261,659.12
VaR Individual Diario al 99% : $370,069.49

## Efecto de Diversificación ##
VaR Individual Anual al 95% : $896,565.32
VaR Individual Anual al 99% : $1,268,029.44
VaR Individual Diario al 95% : $56,478.31
VaR Individual Diario al 99% : $79,878.35

